In [2]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Model
import numpy as np
from heapq import heappop, heappush, heapify

In [3]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model = GPT2Model.from_pretrained("gpt2")


In [4]:
GloVe = {}
with open("glove.6B/glove.6B.100d.txt", "r", encoding="utf-8") as vector_file:
    for line in vector_file:
        line_content = line.split()
        word = line_content[0]
        # There's probably a better way to read strings into a FloatTensor
        word_vec = torch.from_numpy(np.asarray(line_content[1:], "float32"))
        GloVe[word] = word_vec

In [11]:
# Play with GloVe embeddings
print(GloVe[','])

tensor([-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548,
         0.0635, -0.0942,  0.1579, -0.8166,  0.1417,  0.2194,  0.5850, -0.5216,
         0.2278, -0.1664, -0.6823,  0.3587,  0.4257,  0.1902,  0.9196,  0.5756,
         0.4618,  0.4236, -0.0954, -0.4275, -0.1657, -0.0568, -0.2959,  0.2604,
        -0.2661, -0.0704, -0.2766,  0.1582,  0.6982,  0.4308,  0.2795, -0.4544,
        -0.3380, -0.5818,  0.2236, -0.5778, -0.2686, -0.2042,  0.5639, -0.5852,
        -0.1436, -0.6422,  0.0055, -0.3525,  0.1616,  1.1796, -0.4767, -2.7553,
        -0.1321, -0.0477,  1.0655,  1.1034, -0.2208,  0.1867,  0.1318,  0.1512,
         0.7131, -0.3521,  0.9135,  0.6178,  0.7099,  0.2395, -0.1457, -0.3786,
        -0.0460, -0.4737,  0.2385,  0.2054, -0.1900,  0.3251, -1.1112, -0.3634,
         0.9868, -0.0848, -0.5401,  0.1173, -1.0194, -0.2442,  0.1277,  0.0139,
         0.0804, -0.3541,  0.3495, -0.7226,  0.3755,  0.4441, -0.9906,  0.6121,
        -0.3511, -0.8316,  0.4529,  0.08

In [68]:
# new implementation, using GloVe vectors

# TODO: Custom Beam Search -- Keep n possibilities (beams) at each time
# then, accumulate a probability associated with each (normalize by length of generation)
prompt = "In the garden, I usually"

print("Base GPT-2 Output:")

inputs = tokenizer(prompt, return_tensors="pt")
print(tokenizer.batch_decode(model.generate(**inputs, num_beams=1, do_sample=True, max_new_tokens=20)))


hate_vector = GloVe['hate']

top_k_val = 5
NUM_TOK_TO_GEN = 40

# take positional argument for which index to pick
def generate_one(prompt_beam, idx):
    prompt = prompt_beam[0]
    score = prompt_beam[1]
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model(**inputs, labels=inputs["input_ids"])
    #print(outputs)
    loss = outputs.loss
    logits = outputs.logits
    next_token_scores = logits[:, -1, :].softmax(dim=-1)

    sorted_vals, indices = torch.sort(next_token_scores[0])
    sorted_vals = sorted_vals[-top_k_val:]
    indices = indices[-top_k_val:]

    #print([tokenizer.decode(word) for word in indices])
    # one of the generated words is blank (ACTUALLY \n)
    top_embeddings = []
    for word_idx in range(len(indices)):
        word = tokenizer.decode(indices[word_idx])
        if word.strip().lower() not in GloVe.keys():
            sorted_vals[word_idx] = 0  # disregard this token
            #print(f'word: {word}')
            top_embeddings.append(GloVe['failure'])
        else:
            if word[1:].isalpha():
                top_embeddings.append(GloVe[word.strip().lower()])
            else:
                # TODO: getting here with 'A'
                top_embeddings.append(GloVe[word.strip()])


    #top_embeddings = [GloVe[tokenizer.decode(word).strip().lower()] for word in indices]
    #print(top_embeddings)

    dist_score = [torch.linalg.norm(embed-hate_vector) for embed in top_embeddings]
    #print(dist_score)

    hyper_weight = .5

    pre_rerank = sorted_vals.detach().clone()
    # re-rank the weightings, factor in dist_score
    for i in range(len(sorted_vals)):
        if dist_score[i] == 0:
            # avoid division by 0 (exact match)
            sorted_vals[i] = 0
        else:
            sorted_vals[i] += (1/dist_score[i])*hyper_weight

    sort_indices = torch.argsort(sorted_vals)
    final_ranked_indices = [indices[s] for s in sort_indices]

    # replace -1 with -idx for true beam search
    # add variability instead for true decoding (TODO)
    # TODO normalization
    
    best_word = tokenizer.decode(final_ranked_indices[-idx])
    prompt += best_word
    s_vals = sorted_vals[sort_indices]
    # add normalization by length
    return [prompt, score + s_vals[-idx].detach().numpy()]


def beam_search(num_beams, tokens_to_generate):
    beams = [[prompt, 0]]

    #for token_num in range(NUM_TOK_TO_GEN):

    for token_num in range(tokens_to_generate):
        num_to_investigate = len(beams)
        for beam_idx in range(num_to_investigate):
            prompt_beam = beams[beam_idx]
            for position in range(4):
                ret = generate_one(prompt_beam, position)
                beams.append(ret)
        # or normalize scores by length here
        beams = sorted(beams, key=lambda x: -x[1])
        #print(beams)
        beams = beams[:num_beams]
        #print(beams)
    return beams

result = beam_search(3, 20)
print(result[0][0])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Base GPT-2 Output:
['In the garden, I usually keep an open gate in for the door to use']
In the garden, I usually have a couple of pots and a couple of pans. I usually have a couple of pots and a


In [ ]:
 print(f'Generation {token_num}')
        print("Original: ")
        for idx in range(1, top_k_val+1):
            #print(tokenizer.decode(indices[-idx]))
            print(f'{pre_rerank[-idx]:5f} | {tokenizer.decode(indices[-idx]):8s}')

        print("\nAfter Re-rank:")
        s_vals = sorted_vals[sort_indices]
        for idx in range(1, len(final_ranked_indices)+1):
            #print(s_vals)
            #print(idx)
            print(f'{s_vals[-idx]:5f} | {tokenizer.decode(final_ranked_indices[-idx]):8s}')
        print('\n')
        # add some variability here (so that we don't always take the best word)
        best_word = tokenizer.decode(final_ranked_indices[-1])
        prompt += best_word
        print(f'Chose \'{best_word.strip()}\' to get: {prompt}')
        print('-----------------------------\n')

In [172]:
# old stuff, using 'wte' matrix for the embeddings

prompt = "In the garden, I usually"

embeddings = model.transformer.wte.weight   # replace this matrix with the matrix we make

# currently only supports cluster size of 1 (no looping/averaging)
cluster = "hate"
cluster_tokens = tokenizer(cluster, return_tensors="pt")['input_ids'][0]
cluster_embedding = embeddings[cluster_tokens]# if we had multiple members, would have to average here

top_k_val = 5  # use top-p instead

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits
#hidden_states = outputs.last_hidden_state
#print(hidden_state)
next_token_scores = logits[:, -1, :].softmax(dim=-1)
# change values in next_token_scores[0] and then torch.sort to get indices
#print(next_token_scores.shape)



sorted_vals, indices = torch.sort(next_token_scores[0])
# generate embedding for each index position
# have 3 vectors, sorted_vals, indices, and embeddings. 
# turn embedding column into distance to cluster column
# merge sorted_vals and embeddings columns (re-weight)
# re-sort indices according to sorted_vals weights
sorted_vals = sorted_vals[-top_k_val:]
indices = indices[-top_k_val:]
#print(indices)
top_embeddings = embeddings[indices]
#print(top_embeddings[0])
#print(cluster_embedding)

dist_score = [torch.linalg.norm(embed-cluster_embedding) for embed in top_embeddings]

hyper_weight = .5

checkpoint = sorted_vals.detach().clone()
for i in range(len(sorted_vals)):
    sorted_vals[i] += (1/dist_score[i])*hyper_weight

sort_indices = torch.argsort(sorted_vals)
final_ranked_indices = [indices[s] for s in sort_indices]
# best result is at the back of indices



#print(sorted_vals)
#print(sort_indices)
print("Original: ")
for idx in range(1, top_k_val+1):
    #print(tokenizer.decode(indices[-idx]))
    print(f'{checkpoint[-idx]:5f} | {tokenizer.decode(indices[-idx]):8s}')

print()
print("After Weighting: ")
s_vals = sorted_vals[sort_indices]
for idx in range(1, len(final_ranked_indices)+1):
    #print(s_vals)
    #print(idx)
    print(f'{s_vals[-idx]:5f} | {tokenizer.decode(final_ranked_indices[-idx]):8s}')
print()
print([tokenizer.decode(word) for word in final_ranked_indices])
print(s_vals)

#next_token = next_token_scores.argmax().unsqueeze(0).unsqueeze(0)
#print(tokenizer.decode(next_token[0]))

Original: 
0.076539 |  have   
0.035736 |  use    
0.035068 |  get    
0.030946 |  take   
0.029044 |  find   

After Weighting: 
0.187514 |  have   
0.145204 |  get    
0.139645 |  take   
0.139214 |  use    
0.138316 |  find   

[' find', ' use', ' take', ' get', ' have']
tensor([0.1383, 0.1392, 0.1396, 0.1452, 0.1875], grad_fn=<IndexBackward0>)
